In [9]:
# %pip install langchain langchain-aws langchain_core

In [10]:
from langchain_aws import ChatBedrock

In [11]:
llm = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs=dict(temperature=0),
    # other params...
)

In [12]:
# Example tweets and their metadata
raw_tweets = [
    {"id": "001", "text": "It's been a week since the Sparks Valley fire roared to life, and it's now the largest wildfire in state history.", "date": "03-04-2024"},
    {"id": "002", "text": "Video of the flames destroying my neighborhood. **** that fire! #firessuck", "date": "03-04-2024"},
    {"id": "003", "text": "I blame Biden for the #sparksvalleyfire response", "date": "02-29-2024"},
]

In [13]:
from pydantic.v1 import BaseModel, Field, Extra

class Tweet(BaseModel):
    isPolitical: bool = Field(description="Whether the tweet is political")
    isOffensive: bool = Field(description="Whether the tweet is offensive")

    class Config:
        extra = Extra.forbid  # Forbid extra fields not defined in the model

In [14]:
# Note: 
# This example uses Langchain as a basis for interacting with a 
# local Ollama model but conceptually applies to any LLM.



from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser
from pydantic.v1 import ValidationError

screened_tweets = []

parser = JsonOutputParser(pydantic_object=Tweet)

def check_tweet(tweet):
    # Extract tweet text
    tweet_text = tweet['text']
    print(tweet_text)

    # Define a prompt template for assessing tweet content
    # and include the formatting instructions 
    prompt = PromptTemplate(
        template="""
        Assess whether the tweet contains references to politicians or political parties. 
        Assess if the tweet contains offensive language. 
        {format_instructions}
        
        {tweet_text}
        """,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Construct a Langchain Chain to connect the prompt template with the LLM and Pydantic parser
    chain = prompt | llm | parser
    result = chain.invoke({"tweet_text": tweet_text})
    
    print(result)
    print("------")

for tweet in raw_tweets:
    check_tweet(tweet)

It's been a week since the Sparks Valley fire roared to life, and it's now the largest wildfire in state history.
{'isPolitical': False, 'isOffensive': False}
------
Video of the flames destroying my neighborhood. **** that fire! #firessuck
{'isPolitical': False, 'isOffensive': True}
------
I blame Biden for the #sparksvalleyfire response
{'isPolitical': True, 'isOffensive': False}
------
